# Web Crawling Mini Project
## '네이버 중고나라에 게시되어 있는 중고 스노우보드 게시글 제목 크롤링'
- 네이버에 로그인 한 후 네이버 중고나라에 접속한다.
- 개인정보 보호를 위해 로그인은 1회만 실시, 해당 코드의 셀은 더 이상 실행하지 않고 아이디와 비밀번호는 지운다.(Github 공유)
- 해당 게시글의 제목, 작성자, 게시한 시간, 조회수 크롤링
- 스노우보드는 데크, 바인딩, 부츠로 나눠져 있는데, 데크를 판매중인 게시글만 크롤링한다.

### 0. 네이버 중고나라 페이지 탐색
- 중고 스노우보드 관련 게시글은 스포츠/레저의 하위 카테고리 스키/보드/의류에 게시된다.
- 카테고리, 페이지 번호가 넘어갈 때 URL이 바뀌지 않는다.(Selenium 이용)
- 스노우보드 뿐만 아니라 스키, 의류들도 같이 게시되어 있으므로 검색창을 활용한다.(검색어 : 스노우보드, 보드, 스노보드)

### 1. 네이버 로그인
1. 가끔 로그인에 실패했을 떄 다시 실시하면 ID가 겹치는 **오류**
2. 새로고침을 해도 ID와 PW가 clear되지 않는 **오류**

3. 자동문자입력방지로 인해 자동으로 로그인이 안 되는 **오류**
    - 입력속도가 빠르면 로봇으로 인식한다고 판단
    - ID와 PW 입력란에 한 글자씩 천천히 입력
    - for구문과 time.sleep(second)를 이용

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome('Chromedriver')
driver.get('https://naver.com') # 네이버 접속
driver.find_element_by_class_name('ico_local_login').click() # 로그인 페이지에 접속(tag : i)

ID = '------'
PW = '-------'
id_input = driver.find_element_by_name('id')
pw_input = driver.find_element_by_name('pw')

for word in ID:
    id_input.send_keys(word)
    time.sleep(0.3)

for word in PW:
    pw_input.send_keys(word)
    time.sleep(0.3)

driver.find_element_by_class_name('btn_global').click() #tag : input

### 2. 중고나라 접속
- 중고나라 링크로 접속하면 새로운 Tab이 열려서 driver가 새로운 Tab에 대하여 작동하지 않는 **오류.**
    - driver.window_handles를 통해 Tab 선택
    - switch_to.window()로 해결

In [2]:
# 검색창에 중고나라 검색
driver.find_element_by_name('query').send_keys('중고나라') # 검색창에 '중고나라' 입력
driver.find_element_by_id('search_btn').click() # 검색 클릭
driver.implicitly_wait(5) # 페이지 로딩시간 대기

# 중고나라 접속
driver.find_element_by_id('web_layer_0') # '중고나라'링크가 포함되어 있는 HTML부분 찾기 
driver.find_element_by_class_name('title_link').click() # '중고나라'링크 클릭
driver.switch_to.window(driver.window_handles[1]) # 새로 열린 Tab으로 driver 변경

### 3. 스키/보드/의류 카테고리 접속
1. 누루고자 하는 카테고리(스키/보드/의류)의 element는 찾는데 click은 작동하지 않는 **오류**.
    - 여러 시험 끝에 스크롤이 해당 카테고리까지 내려가야 click이 작동된다는 사실을 알아냄.
    - 스크롤을 자동으로 내리는 코드를 작성

#### *스크롤 내리는 방법*
1. 단순히 window height 값을 넣어줘서 내리는 방법
    - driver.execute_script('window.scrollTo(0, 5700)')
2. 바닥까지 내리는 방법
    - driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
3. 내가 원하는 element까지만 내리는 방법
    - driver.find_element_by_id('menuLink435').location_once_scrolled_into_view

**참고https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python**

In [4]:
# Scroll Method 2
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')# 바닥까지 내리는 방법
driver.find_element_by_id('menuLink435').click() # 카테고리(스키/보드/의류) 클릭

### 4. 검색창에 '스노우보드' 검색(검색유형 : 제목만) ------------------------------------- 해결 못 하는 중
1. 검색유형을 '제목만'으로 변경하는 과정에서 해당 element를 찾지 못하는 **오류**
<br>HTML에는 육안으로 보이지만 find_element로는 찾을 수 없음
    - 원인은 iframe으로 인해 찾지 못했던 것
    - switch_to_frame() 함수 이용하여 해결
    - Iframe에 대하여https://whatis.techtarget.com/definition/IFrame-Inline-Frame


2. 검색유형을 (제목+내용)에서 (제목만)으로 변경이 안 되는 오류
 1. 첫 번째 해결방법 검색 
    - 원인은 해당 [select] tagd에서 style attribute가 'Display:None'으로 설정되어 있어서 value 선택이 불가능했음
    - [select] tag에서 'Dispaly:None' 속성을 지우고 원하는 value(제목만)으로 변경
    - 해결하지 못해서 결국 질문함, 참고https://stackoverflow.com/questions/52262430/how-to-select-a-value-from-options-as-per-the-html-through-selenium-and-python/52262586?noredirect=1#comment91486848_52262586
    - 알고보니 시각적으로만 되는 것처럼 보임, 실제 적용이 되진 않았음 >>>>>>>> 실패
 2. 두 번째 해결방법 검색
    - Dropdown을 클릭한 후 html를 확인해보니 select말고 다른 곳에서 같은 option을 또 볼 수 있었음
    - 즉 위에서 찾은 select는 단지 시각적으로 보일 뿐, 진짜 option의 value는 다른 tag에 있었음
    - 따라서 해당 tag를 찾아서 실시하니 해결됐음
    - 참고https://stackoverflow.com/questions/52156388/how-do-i-select-items-from-a-drop-down-box-in-selenium-using-python/52157900#52157900

In [28]:
# #document로 들어가기
iframe = driver.find_elements_by_id('cafe_main')[0] # tag : iframe이 여러개임으로 해당하는 iframe의 id를 이용
driver.switch_to_frame(iframe) # switch_to_frame() 이용하여 #document 속으로 이동

# 검색유형1 클릭한 후 (제목+내용)에서 (제목만)으로 변경하기
from selenium.webdriver.support.ui import Select
driver.find_element_by_name('frmSearch').find_element_by_id('searchByInput838').click()
option = driver.find_elements_by_css_selector('ul.seljs_option li')
for opt in option:
    print(opt.text)

driver.find_element_by_css_selector('ul.seljs_option').find_element_by_xpath('./li[.="제목만"]').click()

# 검색창에 '데크' 검색하기
driver.find_element_by_id('query').send_keys('데크')
driver.find_element_by_class_name('btn-search-green').click()

IndexError: list index out of range

### 5. 게시글 제목, 작성자, 업로드 시간, 조회수 크롤링
 - 크롤링 시 조건을 걸어 내가 필요한 물품(데크)만 판매하는 게시글 수집, 따라서 검색어를 '스노우보드'가 아닌 '데크'로 검색
 - 페이지를 넘겨가면서 크롤링, 10페이지만 시도

1. 검색창에 단어를 입력 한 후 검색 버튼을 누루면 검색창에 입력한 단어가 없어지지 않는 **오류**
    - 다른 단어를 'send_keys'를 이용해 입력해도 기존의 단어로 검색이 된다.
    - 아직 원인은 모르겠음
    
    
2. 제목 부분을 크롤링했을 때 가끔 [1](댓글 달린 갯수)가 같이 크롤링 되는 **오류**
    - 두 개의 class를 모두 만족시키는 tag를 찾아야 함
    - select('tag.class > tag.class')를 통해 뒤에 같이 따라오는 tag를 제거했다.
    
    
3. 업로드 시간과 조회수가 같은 tag와 같은 column을 사용해서 분류할 수 없는 **오류**
    - 짝수index(조회수)와 홀수index(시간)으로 분류
    
    
4. 여러 tag들 한 번에 찾기
    - 참고https://stackoverflow.com/questions/42177826/retrieving-multiple-tags-via-beautifulsoup-css-selector

In [7]:
# 페이지 넘기기
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

In [8]:
titles, uploaders, times_and_clickers = [], [], []
for i in range(10):
    driver.find_element_by_link_text('{}'.format(i+1)).click()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    titles += soup.select('span.aaa a.m-tcol-c')
    uploaders += soup.select('span.wordbreak')
    times_and_clickers += soup.select('td.view-count')

title_list, uploader_list, time_and_clicker_list, time_list, clicker_list = [], [], [] ,[], []


for title, uploader in zip(titles, uploaders):
    title_list.append(title.text.strip())
    uploader_list.append(uploader.text.strip())

    
for time_and_clicker in times_and_clickers:
    time_and_clicker_list.append(time_and_clicker.text.strip())
time_list = time_and_clicker_list[0::2]
clicker_list = time_and_clicker_list[1::2]


import pandas as pd
data = {'title' : title_list, 'uploader' : uploader_list, 'time' : time_list, 'clicker' : clicker_list}
df = pd.DataFrame(data, columns = ['title', 'uploader', 'time', 'clicker'])
df

,title,uploader,time,clicker
0,스노우보드 데크 한정판 볼컴 사자데크 W캠 1시즌사용,폴라베어,00:39,17
1,부산) 1617 011플랫킹 152 사이즈 데크만 팜,howon2,00:24,27
2,(삽니다) 해머데크 허리 255 이상 삽니다,사팔따리,00:22,3
3,"1617스키티즈 저크 티탄, 1718 링크 익스클루시브 프로[덤 오비오 자켓]",cheom2,2018.09.11.,19
4,살로몬 정품 보드 데크(바인딩) 부츠 일체 (가방포함) A급 판매합니다220~230...,kjt4441444,2018.09.11.,23
5,데크(APO 159)+바인딩(유니온포스)+부츠(말라뮤트) 세트로 입문용으로 팝니다....,뚜부,2018.09.11.,91
6,ACC poison 149 데크 + Flow (S)바인딩 판매합니다.,aswer86,2018.09.11.,49
7,버튼 상급 커스텀 플라잉 데크 세트 판매합니다,kjhkjh6914,2018.09.11.,72
8,막데크 부츠 무조건 5만원,boardMania,2018.09.11.,41
9,스노우보드 풀셋 판매,qorgkdudls,2018.09.11.,99


### 발생한 오류들 정리
**1. 로그인에 실패했을 때 해당 페이지를 새로고침해도 ID와 PW에 적혀있는 부분이 없어지지 않음.**

**2. '중고나라'를 검색한 후 사이트를 클릭했을 때 새로운 Tab으로 열려서 다음 코드들이 적용되지 않음.**
- 현재 driver는 처음에 켜진 Tab을 다루고 있어서 새로운 Tab에 대한 권한이 없었다는게 원인.
- driver.window_handles : 현재 켜져있는 tab의 명단 출력(list).
- driver.switch_to.window(driver.window_handles[1])로 해결.

**3. HTML상에는 해당 tag, id, class가 있음에도 불구하고 find_element_by 함수가 적용되지 않음.**

- 중간에 #document 부분이 존재해서 HTML을 더 깊숙히 들어갈 수 없었음.(이유는 모르겠음..)
- 따라서 #document 상위 태그를 찾은 다음 switch_to_frame() 함수 이용하여 해결.

**4. 게시글 검색창 왼쪽에 있는 Dropdown 옵션을 바꿀 수 없음.**
    
- 해당 dropdown에 해당하는 element를 찾아 클릭하는 것 까진 했지만 다른 값으로 바꾸진 못했음.
- 결국 구글링에 실패해서 StackOverFlow에 질문했음 __[link text](https://stackoverflow.com/questions/52262430/how-to-select-a-value-from-options-as-per-the-html-through-selenium-and-python/52262586?noredirect=1#comment91486848_52262586)__
- 답변자가 알려준 방법은 HTML 안에서 내가 찾던 [select] 부분 안에 [style attribute] > Display:none 부분을 지움으로써 오류 해결
- 아마도 Dropdown의 options가 visible하지 않아 바꿀 수 없었던 것이 이유였던거 같음

**5. 검색창에 단어를 입력한 후 검색 버튼을 누루면 해당 단어가 검색창에서 지워지지 않는다.**

- 해당 단어가 없어지지 않아 다른 단어를 입력해도 처음 입력한 단어로 검색이 실행됨
- 아직 원인 불명

**6. 제목 부분을 크롤링했을 때 가끔 [1](댓글 달린 갯수)가 같이 크롤링 된다.**
    - 두 개의 class를 모두 만족시키는 tag를 찾아야 함
    - select('tag.class > tag.class')를 통해 뒤에 같이 따라오는 tag를 제거했다.
    
**7. 업로드 시간과 조회수가 같은 tag와 같은 column을 사용해서 분류할 수 없다.**
    - 짝수index(조회수)와 홀수index(시간)으로 분류